##### Copyright 2025 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini - Multimodal live API: Tool use

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_LiveAPI_tools.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook provides examples of how to use tools with the multimodal live API with [Gemini 2.5](https://ai.google.dev/gemini-api/docs/models/gemini-v2).

The API provides Google Search, Code Execution and Function Calling tools. The earlier Gemini models supported versions of these tools. The biggest change with Gemini 2.5 (in the Live API) is that, basically, all the tools are handled by Code Execution. With that change, you can use **multiple tools** in a single API call, and the model can use multiple tools in a single code execution block.  

This tutorial assumes you are familiar with the Live API, as described in the [this tutorial](../quickstarts/Get_started_LiveAPI.ipynb).

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.5 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../quickstarts/Get_started.ipynb) notebook.

In [17]:
%pip install -U -q google-genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](../quickstarts/Authentication.ipynb) for an example.

In [18]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

### Initialize SDK client

The client will pickup your API key from the environment variable.
To use the live API you need to set the client version to `v1alpha`.

In [19]:
from google import genai

client = genai.Client(http_options={"api_version": "v1alpha"})

### Select a model

Either select the latest stable model or one of the preview ones. Note that the native audio models like `gemini-2.5-flash-native-audio-preview-09-2025` won't work in this notebook as they only can't output text only which is what colab imposes to us.

In [20]:
MODEL_ID = 'gemini-live-2.5-flash-preview'  # @param ['gemini-2.0-flash-live-001', 'gemini-live-2.5-flash-preview', 'gemini-2.5-flash-native-audio-preview-09-2025'] {allow-input: true, isTemplate: true}

### Imports

In [21]:
import asyncio
import contextlib
import json
import wave

from IPython.display import display, Markdown, Audio, HTML

from google import genai
from google.genai import types

### Utilities

You're going to use the Live API's audio output, the easiest way hear it in Colab is to write the `PCM` data out as a `WAV` file:

In [22]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

Use a logger so it's easier to switch on/off debugging messages.

In [23]:
import logging
logger = logging.getLogger('Live')
logger.setLevel('INFO')
#logger.setLevel('DEBUG')  # Switch between "INFO" and "DEBUG" to toggle debug messages.

## Get started

Most of the Live API setup will be similar to the [starter tutorial](../quickstarts/Get_started_LiveAPI.ipynb). Since this tutorial doesn't focus on the realtime interactivity of the API, the code has been simplified: This code uses the Live API, but it only sends a single text prompt, and listens for a single turn of replies.

You can set `modality="AUDIO"` on any of the examples to get the spoken version of the output.

In [24]:
n = 0
async def run(prompt, modality="TEXT", tools=None):
  global n
  if tools is None:
    tools=[]

  config = {
          "tools": tools,
          "response_modalities": [modality]
  }

  async with client.aio.live.connect(model=MODEL_ID, config=config) as session:
    display(Markdown(prompt))
    display(Markdown('-------------------------------'))
    await session.send_client_content(
      turns={"role": "user", "parts": [{"text": prompt}]}, turn_complete=True
    )

    audio = False
    filename = f'audio_{n}.wav'
    with wave_file(filename) as wf:
      async for response in session.receive():
        logger.debug(str(response))
        if response.server_content and response.server_content.model_turn and response.server_content.model_turn.parts and hasattr(response.server_content.model_turn.parts[0], 'text'):
          if text := response.server_content.model_turn.parts[0].text:
            display(Markdown(text))
            continue

        if response.server_content and response.server_content.model_turn and response.server_content.model_turn.parts and hasattr(response.server_content.model_turn.parts[0], 'data'):
          if data := response.server_content.model_turn.parts[0].data:
            print('.', end='')
            wf.writeframes(data)
            audio = True
            continue

        server_content = response.server_content
        if server_content is not None:
          handle_server_content(wf, server_content)
          continue

        tool_call = response.tool_call
        if tool_call is not None:
          await handle_tool_call(session, tool_call)


  if audio:
    display(Audio(filename, autoplay=True))
    n = n+1

Since this tutorial demonstrates several tools, you'll need more code to handle the different types of objects it returns.

- The `code_execution` tool can return `executable_code` and `code_execution_result` parts.
- The `google_search` tool may attach a `grounding_metadata` object.

In [25]:
def handle_server_content(wf, server_content):
  model_turn = server_content.model_turn
  if model_turn:
    for part in model_turn.parts:
      executable_code = part.executable_code
      if executable_code is not None:
        display(Markdown('-------------------------------'))
        display(Markdown(f'``` python\n{executable_code.code}\n```'))
        display(Markdown('-------------------------------'))

      code_execution_result = part.code_execution_result
      if code_execution_result is not None:
        display(Markdown('-------------------------------'))
        display(Markdown(f'``` \n{code_execution_result.output}\n```'))
        display(Markdown('-------------------------------'))

  grounding_metadata = getattr(server_content, 'grounding_metadata', None)
  if grounding_metadata is not None:
    display(
        HTML(grounding_metadata.search_entry_point.rendered_content))

  return

- Finally, with the `function_declarations` tool, the API may return `tool_call` objects. To keep this code minimal, the `tool_call` handler just replies to every function call with a response of `"ok"`.

In [26]:
async def handle_tool_call(session, tool_call):
  print("Tool call:")
  function_responses = []
  for fc in tool_call.function_calls:
    function_response = types.FunctionResponse(
        id=fc.id,
        name=fc.name,
        response={"result": "ok"},
    )
    function_responses.append(function_response)
  print('>>> ', function_responses)
  await session.send_tool_response(function_responses=function_responses)

Try running it for a first time:

In [27]:
await run(prompt="Hello?", tools=None, modality = "TEXT")

ConnectionClosedError: received 1008 (policy violation) models/gemini-live-2.5-flash-preview is not found for API version v1alpha, or is not supported for bidiGenerateContent. Cal; then sent 1008 (policy violation) models/gemini-live-2.5-flash-preview is not found for API version v1alpha, or is not supported for bidiGenerateContent. Cal

## Simple function call

The function calling feature of the API Can handle a wide variety of functions. Support in the SDK is still under construction. So keep this simple just send a minimal function definition: Just the function's name.

Note that in the live API function calls are independent of the chat turns. The conversation can continue while a function call is being processed.

In [ ]:
turn_on_the_lights = {'name': 'turn_on_the_lights'}
turn_off_the_lights = {'name': 'turn_off_the_lights'}

In [ ]:
prompt = "Turn on the lights"

tools = [
    {'function_declarations': [turn_on_the_lights, turn_off_the_lights]}
]

await run(prompt, tools=tools, modality = "TEXT")

## Async function calling

**Async function calling** lets the model manage its function calls asynchronously and without blocking the user input.

You can decide how the model will behave when the function call ends between saying nothing, interrupting what it's doing or waiting to finish its current task.

The next cells are going to use a slightly updated code to use the Live API so that the session stays open for 20s and accepts multiple requests that are sent to the model every 10s. Expand the next cell if you are curious about this implementation.

In [ ]:
# @title Live class with multiple messages (just run this cell)

import collections.abc
import inspect
from asyncio.exceptions import CancelledError
import traceback

class Live:
  def __init__(self, client):
    self.client = client


  async def run(self, config, functions=None, messages=None):
    self.config = config
    self.send_queue = asyncio.Queue()
    self.tool_call_queue = asyncio.Queue()

    try:
      async with (
            client.aio.live.connect(model=MODEL_ID, config=config) as session,
            asyncio.TaskGroup() as tg
      ):
        self.session = session
        recv_task = tg.create_task(self._recv())
        send_task = tg.create_task(self._send())
        tool_call_task = tg.create_task(self._run_tool_calls(functions))
        read_text= tg.create_task(self._read_text(messages))


        await read_text
        await asyncio.sleep(20) # Keeping the socket open for 20s to wait for the FC and different messages

        raise CancelledError
    except CancelledError:
      pass
    except ExceptionGroup as EG:
      traceback.print_exception(EG)

  async def _recv(self):
    try:
      mode = None
      while True:
        async for response in self.session.receive():
          logger.debug(str(response))
          if response.server_content and response.server_content.model_turn and response.server_content.model_turn.parts and hasattr(response.server_content.model_turn.parts[0], 'text'):
            if text := response.server_content.model_turn.parts[0].text:
              if mode != 'text':
                mode = 'text'
                print()
              print(text)
          else:
            if mode == 'text':
              mode = 'other'
              print()
            print(f'<<<  {response.model_dump_json(exclude_none=True)}\n')

          tool_call = response.tool_call
          if tool_call is not None:
            await self.tool_call_queue.put(tool_call)

    except asyncio.CancelledError:
      pass

  async def _send(self):
    while True:
      msg = await self.send_queue.get()
      print(f'>>> {repr(msg)}\n')
      await self.session.send_client_content(turns=msg,turn_complete=True)

  async def _run_tool_calls(self, functions):
    while True:
      tool_call = await self.tool_call_queue.get()
      for fc in tool_call.function_calls:
        fun = functions[fc.name]
        called = fun(**fc.args)
        if inspect.iscoroutine(called):
          print(f'>> Starting {fc.name}\n')
          result = await called
          print(f'>> Done {fc.name} >>> {repr(result)}\n')
          result = self._wrap_function_result(fc, result)
          await self.session.send_tool_response(function_responses=[result])
        elif isinstance(called, collections.abc.AsyncIterable):
          async for result in called:
            result.will_continue=True
            result = self._wrap_function_result(fc, result)
            print(f">>> {repr(result)}\n")
            await self.session.send_tool_response(function_responses=[result])

          result = self._wrap_function_result(
              fc,
              types.FunctionResponse(will_continue=False)
          )
          print(f">>> {repr(result)}\n")
          await self.session.send_tool_response(
              function_responses=[result]
          )


        else:
          raise TypeError(f"expected {fc.name} to return a coroutine, or an "
                          f"AsyncIterable, got {type(fun)}")

  def _wrap_function_result(self, fc, result):
    if result is None:
      return types.FunctionResponse(
          name=fc.name,
          id=fc.id,
          response={'result': 'ok'}
      )
    elif isinstance(result, types.FunctionResponse):
      result.name = fc.name
      result.id = fc.id
      return result
    else:
      return types.FunctionResponse(
          name=fc.name,
          id=fc.id,
          response= {'result': result}
      )

  async def _read_text(self, messages):
    if messages:
        for n, message in enumerate(messages):
            await self.send_queue.put({
                'role': 'user',
                'parts': [{'text': message}]
            })
            if n+1 < len(messages):
              await asyncio.sleep(5)
    else:
        while True:
            message = await asyncio.to_thread(input, "message > ")
            if message.lower() == "q":
                break
            await self.send_queue.put({
                'role': 'user',
                'parts': [{'text': message}]
            })

### Default behavior: Blocking

Let's start with the default behavior. First define a mock weather function that simulates compute time by waiting 10s.

The default behavior functions as a FIFO queue: the function call is added to a queue, and any subsequent requests are queued (blocked) behind it until it finishes processing.

In [ ]:
# Mock function, takes 10s to process
async def get_weather_vegas():
  await asyncio.sleep(10)
  return {'weather': "Sunny, 42 degrees"}

# multiple prompts, they are going to be asked with 5s delay between each of them.
questions = [
    "What's the weather in Vegas?",
    "In the meantime tell me about the Paris casino"
]

await Live(client).run(
    messages=questions,
    functions={
        'get_weather_vegas': get_weather_vegas,
    },
    config={
        "response_modalities": ["TEXT"],
        "tools": [
            {
                'function_declarations': [
                    {'name': 'get_weather_vegas',  "behavior": "UNSPECIFIED"}, # This is default behavior, equivalent to BLOCKING
                ]
            }
        ]
    }
)

As you can see, the model called the `get_weather_vegas` function right away, but then the second question was ignored as the model was still waiting for the function call results. It only started to answer the second question after answering the function call.

### **Interrupt**: stop what you're doing and handle this result

This time, `behavior` is set as `NON_BLOCKING`, which means it will use async function calling.

When you do, you need to define what the model will do when it will get the result of the function call. This is managed inside of the function, or within your script that handles the funcion calls (since Automatic function calling is not available) by adding a `scheduling` value in the `FunctionResponse`.

This time the `scheduling` behavior is "**`Interrupt`**", which means that as soon as it gets a response, the model will stop what it's saying and process the response right away.

In [ ]:
# Mock function, takes 10s to process
async def get_weather_vegas():
  await asyncio.sleep(10)
  return types.FunctionResponse(
      response={'weather': "Sunny, 42 degrees"},
      scheduling="INTERRUPT"
  )

# multiple prompts, they are going to be asked with 5s delay between each of them.
questions = [
    "What's the weather in Vegas?",
    "In the meantime tell me what you know about the Paris casino and all there's to do and see in it. Then continue to tell me about the Vegas casinos until I tell you to stom talking. Don't ask me, just talk non-stop"
    "Then can you tell me what's your favorite cirque du soleil show?"
]

await Live(client).run(
    messages=questions,
    functions={
        'get_weather_vegas': get_weather_vegas,
    },
    config={
        "response_modalities": ["TEXT"],
        "tools": [
            {
                'function_declarations': [
                    {'name': 'get_weather_vegas',  "behavior": "NON_BLOCKING"},
                ]
            }
        ]
    }
)

As you can see, this time, the model acknowledged our request by saying something like "`The weather in Vegas request is running. I'll let you know when it's done`", then continues to process what you asked it, and then when the function response comes back, it stopped what it was doing, told us about the weather, and then continued to talk about what it was talking about.

### **Wait until Idle**: Finish what you're doing before handling this result

Once again, the `behavior` is set as `NON_BLOCKING`, which means it will use async function calling and you will have to add a `scheduling` value in the `FunctionResponse`.

This time the `scheduling` behavior is "**`When_idle`**", which means that the model will **wait until it's finished** with what it's saying and only then tell us about what you asked for.

In [ ]:
import time

# Mock function, takes 6s to process
async def get_weather_vegas():
  await asyncio.sleep(6)
  return types.FunctionResponse(
      response={'weather': "Sunny, 42 degres"},
      scheduling="WHEN_IDLE"
  )

# multiple prompts, they are going to be asked with 5s delay between each of them.
questions = [
    "What's the weather in Vegas?",
    "In the meantime, without using tools, tell me what you know about the Paris casino and all there's to do and see in it. Tell me about each casino on the strip!"
]

await Live(client).run(
    messages=questions,
    functions={
        'get_weather_vegas': get_weather_vegas,
    },
    config={
        "response_modalities": ["TEXT"],
        "tools": [
            {
                'function_declarations': [
                    {'name': 'get_weather_vegas',  "behavior": "NON_BLOCKING"},
                ]
            }
        ]
    }
)

As you can see, this time, even though it received the function call response while it was answering about the casinos (cf. `>> Done get_weather_vegas >>> [...] response={'weather': 'Sunny, 42 degres'})` line), it waited until it was finished with its current answer before telling about the weather.

### Silent: Just keep what you learned for yourself

This time again, the `behavior` is set as `NON_BLOCKING`, which means it will use async function calling and will need a `scheduling` value in the `FunctionResponse`.

This time the `scheduling` behavior is "**`Silent`**", which means that the model won't tell you when the function call is finished but it might still use that knowledge later on in the conversation.

In [ ]:
import time

# Mock function, takes 5s to process
async def get_weather_vegas():
  time.sleep(10)
  return types.FunctionResponse(
      response={'weather': "Sunny, 42 degres"},
      scheduling="SILENT"
  )

# multiple prompts, they are going to be asked with 5s delay between each of them.
questions = [
    "What's the weather in Vegas?",
    "In the meantime tell me about the Paris casino.",
    "Is the temperature over 40 degres?"
]

await Live(client).run(
    messages=questions,
    functions={
        'get_weather_vegas': get_weather_vegas,
    },
    config={
        "response_modalities": ["TEXT"],
        "tools": [
            {
                'function_declarations': [
                    {'name': 'get_weather_vegas',  "behavior": "NON_BLOCKING"},
                ]
            }
        ]
    }
)

This time, as you can see, the model did nothing when the function call ended, but when asked again about the same thing it answered without doing a new function call.

## Code execution

The `code_execution` lets the model write and run python code. Try it on a math problem the model can't solve from memory:

In [ ]:
prompt="Can you compute the largest prime palindrome under 100000."

tools = [
    {'code_execution': {}}
]

await run(prompt, tools=tools, modality="TEXT")

## Compositional Function Calling

Compositional function calling refers to the ability to combine user defined functions with the `code_execution` tool. The model will write them into larger blocks of code, and then pause execution while it waits for you to send back responses for each call.


In [ ]:
prompt="Can write some code to loop through and print integers from 1-20, and every time you hit a multiple of 3 turn on the lights, and every time you hit a multiple of 5 turn them off?"

tools = [
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights, turn_off_the_lights]}
]

await run(prompt, tools=tools, modality="TEXT")

## Google search

The `google_search` tool lets the model conduct google searches. For example, try asking it about events that are too recent to be in the training data.

The search will still execute in `AUDIO` mode, but you won't see the detailed results:

In [ ]:
prompt="When the latest Brazil vs. Argentina soccer match happened and what was the final score?"

tools = [
   {'google_search': {}}
]

await run(prompt, tools=tools, modality="TEXT")

## Multi-tool


The biggest difference with the new API however is that you're no longer limited to using 1-tool per request. Try combining those tasks from the previous sections:

In [ ]:
prompt = """\
  Hey, I need you to do three things for me.

  1. Then compute the largest prime plaindrome under 100000.
  2. Then use google search to lookup unformation about the largest earthquake in california the week of Dec 5 2024?
  3. Turn on the lights

  Thanks!
  """

tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': [turn_on_the_lights, turn_off_the_lights]}
]

await run(prompt, tools=tools, modality="TEXT")

## Next Steps

- For more information about the SDK see the [SDK docs](https://googleapis.github.io/python-genai/)
- This tutorial uses the high level SDK, if you're interested in the lower-level details, try the [Websocket version of this tutorial](../quickstarts/websockets/Get_started_LiveAPI_tools.ipynb)
- This tutorial only covers _basic_ usage of these tools for deeper (and more fun) example see the [Search tool tutorial](./Search_Grounding.ipynb)

Or check the other Gemini 2.5 capabilities from the [Cookbook](../gemini-2/), in particular this other [multi-tool](../examples/LiveAPI_plotting_and_mapping.ipynb) example and the one about Gemini [spatial capabilities](../quickstarts/Spatial_understanding.ipynb).